In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
import xlsxwriter
import os
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
X_train = []
X_test = []
for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5", "gap_VGG19.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

In [6]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)
model.save_weights('dog_cat.h5')

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s - loss: 0.2278 - acc: 0.9158 - val_loss: 0.0357 - val_acc: 0.9880
Epoch 2/10
20000/20000 [==============================] - 1s - loss: 0.0409 - acc: 0.9847 - val_loss: 0.0279 - val_acc: 0.9908
Epoch 3/10
20000/20000 [==============================] - 1s - loss: 0.0281 - acc: 0.9903 - val_loss: 0.0264 - val_acc: 0.9914
Epoch 4/10
20000/20000 [==============================] - 1s - loss: 0.0242 - acc: 0.9922 - val_loss: 0.0257 - val_acc: 0.9920
Epoch 5/10
20000/20000 [==============================] - 1s - loss: 0.0194 - acc: 0.9936 - val_loss: 0.0250 - val_acc: 0.9930
Epoch 6/10
20000/20000 [==============================] - 1s - loss: 0.0184 - acc: 0.9941 - val_loss: 0.0260 - val_acc: 0.9920
Epoch 7/10
20000/20000 [==============================] - 1s - loss: 0.0176 - acc: 0.9941 - val_loss: 0.0252 - val_acc: 0.9924
Epoch 8/10
20000/20000 [==============================] - 1s -

In [4]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)
arranged_arr = np.zeros(len(test_generator.filenames))
list = os.listdir('test/test')
list.sort()
for i in range(len(list)):
    num = int(list[i].split('.')[0])
    arranged_arr[num-1] = y_pred[i]

12032/12500 [===========================>..] - ETA: 0sFound 12500 images belonging to 1 classes.


In [5]:
workbook = xlsxwriter.Workbook('pred.xlsx')
worksheet = workbook.add_worksheet()
for i in range(len(arranged_arr)):
    worksheet.write(i,1,arranged_arr[i])
workbook.close()